In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import re
from geopy.geocoders import Nominatim
import shapely
import pyproj
import math
import folium
import requests
import pandas as pd
import time
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

import warnings
warnings.filterwarnings('ignore')

In [ ]:
city_venues= pd.read_csv('/kaggle/input/munich-trending-venues-from-foursquare/city_venues_munich.csv')
city_venues.head()

In [ ]:
city_venues.groupby('Venue Category').count()

In [ ]:
print('There are {} unique categories.'.format(len(city_venues['Venue Category'].unique())))

There are 184 unique categories.

Analyze Each Property's surrounding area - using one hot coding, each venue category will become a column in the dataframe:

In [ ]:
# one hot encoding
city_venues_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

# add property column back to dataframe
city_venues_onehot['Neighborhood'] = city_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [city_venues_onehot.columns[-1]] + list(city_venues_onehot.columns[:-1])
city_venues_onehot = city_venues_onehot[fixed_columns]

city_venues_onehot.head()

In [ ]:
city_venues_onehot.shape

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category:

In [ ]:
city_venues_grouped = city_venues_onehot.groupby('Neighborhood').mean().reset_index()
city_venues_grouped

In [ ]:
city_venues_grouped.shape

In [ ]:
num_top_venues = 7

for hood in city_venues_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = city_venues_grouped[city_venues_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
um_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['Neighborhood'] = city_venues_grouped['Neighborhood']

for ind in np.arange(city_venues_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_venues_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted